# Text Mining project
 
 Partie 2 : réalisation d'un modèle de prédiction des tweets positifs ou négatifs
 
 thème : confinement suite au covid_19
 
 Hashtag choisi : #Lockdown2020


In [1]:
import re
import nltk
import numpy as np

In [2]:
from sklearn.datasets import load_files
nltk.download('stopwords')

import pickle
from nltk.corpus import stopwords

stopwords.words('english')[0:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [3]:
#import datasets
movie_data = load_files("lockdown2020")
X, y = movie_data.data, movie_data.target

In [4]:
movie_data

{'data': [b'Lockdown has made me so self-sufficient that I forget how to socialise.\r\n#lockdownextension \r\n#lockdown2020 \r\n#LockdownEnd \r\n#QuarantineLife',
  b"Best part of 9hrs later and I've completed 12 masks for my family, just another 14 or so to do tomorrow. I knew that GCSE in Textiles would finally come in use \xf0\x9f\x98\x82 #lockdown2020 #coronavirus #staysafe https://t.co/pm9zPQl58p",
  b'@Jackson_Carlaw  Where\xe2\x80\x99s your statement about bojos right hand man then? Can\xe2\x80\x99t be having one rule for him and a rule for everyone else. #Hypocrite #DominicCummings #COVID\xe3\x83\xbc19 #lockdown2020 https://t.co/RbaU3JiBoZ',
  b'It is not easy working for the ignorant and the ungrateful.\r\n#EidMubarak \r\n#EidUlFitr \r\n#staysafe \r\n#\xd8\xa7\xd9\x84\xd9\x82\xd8\xa8\xd8\xb6_\xd8\xb9\xd9\x84\xd9\x89_\xd8\xa7\xd9\x84\xd9\x85\xd8\xba\xd8\xaa\xd8\xb5\xd8\xa8_\xd9\x85\xd8\xa7\xd8\xb2\xd9\x86_\xd8\xa7\xd8\xa8\xd8\xb1\xd8\xa7\xd9\x87\xd9\x8a\xd9\x85 \r\n#SaturdayTho

In [5]:
len(y)

171

In [6]:
X[1]

b"Best part of 9hrs later and I've completed 12 masks for my family, just another 14 or so to do tomorrow. I knew that GCSE in Textiles would finally come in use \xf0\x9f\x98\x82 #lockdown2020 #coronavirus #staysafe https://t.co/pm9zPQl58p"

In [7]:
#preprocessing
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
document = X[1]
document

b"Best part of 9hrs later and I've completed 12 masks for my family, just another 14 or so to do tomorrow. I knew that GCSE in Textiles would finally come in use \xf0\x9f\x98\x82 #lockdown2020 #coronavirus #staysafe https://t.co/pm9zPQl58p"

In [9]:
document = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(document))
document

'b Best part of 9hrs later and I ve completed 12 masks for my family  just another 14 or so to do tomorrow  I knew that GCSE in Textiles would finally come in use  xf0 x9f x98 x82  lockdown2020  coronavirus  staysafe  '

In [10]:
#enlever les caractères spéciaux, y compris la ponctuation
document = re.sub(r'\W', ' ', document)
document

'b Best part of 9hrs later and I ve completed 12 masks for my family  just another 14 or so to do tomorrow  I knew that GCSE in Textiles would finally come in use  xf0 x9f x98 x82  lockdown2020  coronavirus  staysafe  '

In [11]:
# enlever les mots à un caractère
document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
document

'b Best part of 9hrs later and ve completed 12 masks for my family  just another 14 or so to do tomorrow knew that GCSE in Textiles would finally come in use  xf0 x9f x98 x82  lockdown2020  coronavirus  staysafe  '

In [12]:
# Remplacer les espaces multiples par un espace
document = re.sub(r'\s+', ' ', document, flags=re.I)
document

'b Best part of 9hrs later and ve completed 12 masks for my family just another 14 or so to do tomorrow knew that GCSE in Textiles would finally come in use xf0 x9f x98 x82 lockdown2020 coronavirus staysafe '

In [13]:
# Enlever la lettre b au début de chaque commentaire
document = re.sub(r'^b\s+', '', document)
document

'Best part of 9hrs later and ve completed 12 masks for my family just another 14 or so to do tomorrow knew that GCSE in Textiles would finally come in use xf0 x9f x98 x82 lockdown2020 coronavirus staysafe '

In [14]:
 #enlever les  x99 x82 .... qui sont présents dans plusieurs tweets
document = re.sub(r'[x][\w][\w]','',document)
document

'Best part of 9hrs later and ve completed 12 masks for my family just another 14 or so to do tomorrow knew that GCSE in Teles would finally come in use     lockdown2020 coronavirus staysafe '

In [15]:
# Convertir en minuscules
document = document.lower()
document

'best part of 9hrs later and ve completed 12 masks for my family just another 14 or so to do tomorrow knew that gcse in teles would finally come in use     lockdown2020 coronavirus staysafe '

In [16]:
# Lemmatisation/stemming
document = document.split()

document = [stemmer.lemmatize(word) for word in document]
document = ' '.join(document)
document

'best part of 9hrs later and ve completed 12 mask for my family just another 14 or so to do tomorrow knew that gcse in teles would finally come in use lockdown2020 coronavirus staysafe'

In [17]:
documents = []

for sen in range(0, len(X)):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    #removing x**
    document = re.sub(r'[x][\w][\w]','',document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

#To see the number of features
vectorizer = CountVectorizer(min_df=4, max_df=0.6, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()
print(len(X[0]))
print(X)
feature_list = vectorizer.get_feature_names()
feature_list

105
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


['10',
 '19',
 'also',
 'amp',
 'away',
 'back',
 'best',
 'case',
 'come',
 'coronavirus',
 'could',
 'country',
 'covid',
 'covid19',
 'covid_19',
 'cummings',
 'day',
 'died',
 'difficult',
 'dominiccummings',
 'eid',
 'eidmubarak',
 'end',
 'everyone',
 'family',
 'feel',
 'feeling',
 'finally',
 'first',
 'food',
 'free',
 'friend',
 'get',
 'go',
 'going',
 'good',
 'got',
 'government',
 'great',
 'ha',
 'haircut',
 'happy',
 'home',
 'jisoo',
 'kid',
 'know',
 'life',
 'like',
 'lockdown',
 'lockdownhouseparty',
 'lockdownlife',
 'lovely',
 'made',
 'making',
 'month',
 'must',
 'need',
 'new',
 'news',
 'ngladbach',
 'nharvetz',
 'nhttps',
 'ni',
 'night',
 'nsikkim',
 'nsope',
 'nthis',
 'one',
 'open',
 'pandemic',
 'people',
 'quarantine',
 'quarantinelife',
 'right',
 'rule',
 'saturday',
 'saturdaythoughts',
 'see',
 'shopping',
 'since',
 'socialdistancing',
 'sonusood',
 'soon',
 'standwithjamia',
 'stay',
 'stayhome',
 'staysafe',
 'still',
 'story',
 'sunday',
 'thank

In [19]:
#Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selector = SelectKBest(chi2, k=100)

X_new = selector.fit_transform(X, y)
X_new

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [20]:
cols = selector.get_support(indices=True)
cols
for i in cols :
    print(feature_list[i])

10
19
also
amp
away
back
best
case
come
coronavirus
could
country
covid
covid19
covid_19
cummings
day
died
difficult
dominiccummings
eid
eidmubarak
end
family
feeling
finally
first
food
free
friend
get
go
going
good
got
government
great
ha
haircut
happy
home
jisoo
kid
know
life
like
lockdown
lockdownhouseparty
lockdownlife
lovely
made
making
must
need
new
news
ngladbach
nharvetz
ni
night
nsikkim
nsope
nthis
one
open
pandemic
people
quarantinelife
right
rule
saturday
saturdaythoughts
see
shopping
since
socialdistancing
sonusood
soon
standwithjamia
stay
stayhome
staysafe
still
story
sunday
thanks
thing
think
time
today
tomorrow
two
video
wa
want
week
weekend
well
would
year


In [84]:
#max_features : top max_features ordered by term frequency across the corpus
vectorizer = CountVectorizer(max_features=1500, min_df=4, max_df=0.6, stop_words=stopwords.words('english'))  
X = vectorizer.fit_transform(documents).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
len(X)

171

In [104]:
#Training and Testing Sets

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [105]:
#On a choisi de travailler avec RandomizedSearchCV qui nous permet de simuler sur plusieurs paramètres de randomforrest, et choisi vers la fin les meilleurs paramètres
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestClassifier() 
random_grid = {'bootstrap': [True, False],
               'max_depth': [int(x) for x in np.linspace(start = 200, stop = 3000, num = 20)],
               'max_features': ['auto', 'sqrt'],
               'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000,2500,3000,3500]
               }
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, cv = 3, verbose=1, random_state=42, n_jobs = -1)
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   16.1s finished
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [106]:
#Training Text Classification Model and Predicting Sentiment

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=3000, random_state=0)  
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [107]:
#Predict on test set
y_pred_grid =rf_random.predict(X_test)
print("prédiction du modèle rf_random: ",y_pred_grid)
y_pred =classifier.predict(X_test)
print("prédiction du modèle classifier:",y_pred)
print("        y_test:                 ",y_test)
y_pred_logreg = logreg_cv.predict(X_test)
print("deja on peut remarquer que le modèle rf_random prédit mieux que le modèle classifier")

prédiction du modèle rf_random:  [0 0 1 0 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 0 1 0 1 0 1]
prédiction du modèle classifier: [0 1 1 0 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1]
        y_test:                  [0 0 1 1 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 1 1 0 1]
deja on peut remarquer que le modèle rf_random prédit mieux que le modèle classifier


In [108]:
#Evaluating the Model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


print ("metriques du modèle du TP random forrest classifier")
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("l'accuracy du modèle dèja utilisé dans le TP d'avant : " ,accuracy_score(y_test, y_pred))
print ("metriques du modèle qu'on a choisi pour ce travail qu'on a appelé rf_random (pour choisir les hyperparametres optimaux automatiquement)")
print(confusion_matrix(y_test,y_pred_grid))
print(classification_report(y_test,y_pred_grid))
print("l'accuracy de notrte modèle : " ,accuracy_score(y_test, y_pred_grid))
print("on peut aussi afficher les meilleurs paramètres de randomforrest classifer que notre modèle a choisi :")
print("meilleurs paramètres",rf_random.best_params_)
print("meilleurs estimateurs",rf_random.best_estimator_)


metriques du modèle du TP random forrest classifier
[[12  3]
 [ 3 17]]
              precision    recall  f1-score   support

           0       0.80      0.80      0.80        15
           1       0.85      0.85      0.85        20

    accuracy                           0.83        35
   macro avg       0.82      0.82      0.83        35
weighted avg       0.83      0.83      0.83        35

l'accuracy du modèle dèja utilisé dans le TP d'avant :  0.8285714285714286
metriques du modèle qu'on a choisi pour ce travail qu'on a appelé rf_random (pour choisir les hyperparametres optimaux automatiquement)
[[14  1]
 [ 3 17]]
              precision    recall  f1-score   support

           0       0.82      0.93      0.87        15
           1       0.94      0.85      0.89        20

    accuracy                           0.89        35
   macro avg       0.88      0.89      0.88        35
weighted avg       0.89      0.89      0.89        35

l'accuracy de notrte modèle :  0.885714285714

In [36]:
#Save the model
with open('text_classifier', 'wb') as picklefile:  
    pickle.dump(rf_random,picklefile)

In [37]:
#To load the model
with open('text_classifier', 'rb') as training_model:  
    model = pickle.load(training_model)
model

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [98]:
y_pred2 = model.predict(X_test)

print(confusion_matrix(y_test, y_pred2))  
print(classification_report(y_test, y_pred2))  
print(accuracy_score(y_test, y_pred2))

[[14  1]
 [ 3 17]]
              precision    recall  f1-score   support

           0       0.82      0.93      0.87        15
           1       0.94      0.85      0.89        20

    accuracy                           0.89        35
   macro avg       0.88      0.89      0.88        35
weighted avg       0.89      0.89      0.89        35

0.8857142857142857
